<a href="https://colab.research.google.com/github/OhOmah/F1Revisited/blob/main/Pitstop_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PURPOSE OF THIS NOTEBOOK

This notebook serves as the initial analysis to see which visualizations would work best for the pitstop dashboard. I want the dashboard to:
1. Allow a user to pick a driver/race
2. Generate a dashboard showcasing their pace over a race stint compared to other drivers/teammate
3. Share pitstop time, compare to average pitstop time at that track
4. Show potential finishing place if pitstop was within the average time vs. if it was the fastest time
5.

In [ ]:
# Install the needed libaries}
!pip install dash
!pip install plotly
!pip install plotly.express

In [ ]:
# Import the libaries I need
import pandas as pd
import numpy as np
import dash

# Test if it's running in a colab notebook or locally
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [ ]:
# Import the data I need
if IN_COLAB == True:
  print("in colab")
  # Pitstop times
  pitstop = pd.read_csv('/content/drive/MyDrive/F1 Dataset/archive.zip (Unzipped Files)/pit_stops.csv')
  # Lap times
  laptimes = pd.read_csv('/content/drive/MyDrive/F1 Dataset/archive.zip (Unzipped Files)/lap_times.csv')
  # Finishing results
  results = pd.read_csv('/content/drive/MyDrive/F1 Dataset/archive.zip (Unzipped Files)/results.csv')
  # Driver information
  drivers = pd.read_csv('/content/drive/MyDrive/F1 Dataset/archive.zip (Unzipped Files)/drivers.csv')
  # Race information
  race = pd.read_csv('/content/drive/MyDrive/F1 Dataset/archive.zip (Unzipped Files)/races.csv')
  print('Files imported successfully')

else:
  print("not in colab")
  # Pitstop times
  pitstop = pd.read_csv('pit_stops.csv')
  # Lap times
  laptimes = pd.read_csv('lap_times.csv')
  # Finishing results
  results = pd.read_csv('results.csv')
  # Driver information
  drivers = pd.read_csv('drivers.csv')
  # Race information
  race = pd.read_csv('races.csv')
  print("Files imported successfully")


in colab
Files imported successfully


## Log what data I need so I can get it ready for the dashboard.



In [ ]:
pitstop.head()

,raceId,driverId,stop,lap,time,duration,milliseconds
0,841,153,1,1,17:05:23,26.898,26898
1,841,30,1,1,17:05:52,25.021,25021
2,841,17,1,11,17:20:48,23.426,23426
3,841,4,1,12,17:22:34,23.251,23251
4,841,13,1,13,17:24:10,23.842,23842


In [ ]:
drivers.head()

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen


In [ ]:
# Add drivers to pitstops, we only want to get their forename and surname
print(pitstop.shape)
pitstop = pitstop.merge(drivers[['driverId', 'forename', 'surname']])
print(pitstop.shape)
pitstop.head()

(10089, 7)
(10089, 9)


,raceId,driverId,stop,lap,time,duration,milliseconds,forename,surname
0,841,153,1,1,17:05:23,26.898,26898,Jaime,Alguersuari
1,841,153,2,17,17:31:06,24.463,24463,Jaime,Alguersuari
2,841,153,3,35,17:59:45,26.348,26348,Jaime,Alguersuari
3,842,153,1,16,16:32:25,23.584,23584,Jaime,Alguersuari
4,842,153,2,32,17:00:55,22.888,22888,Jaime,Alguersuari


In [ ]:
# Add some general race information
print(pitstop.shape)
pitstop = pitstop.merge(race[['raceId', 'year', 'name']])
print(pitstop.shape)
pitstop.head()

(10089, 9)
(10089, 11)


,raceId,driverId,stop,lap,time,duration,milliseconds,forename,surname,year,name
0,841,153,1,1,17:05:23,26.898,26898,Jaime,Alguersuari,2011,Australian Grand Prix
1,841,153,2,17,17:31:06,24.463,24463,Jaime,Alguersuari,2011,Australian Grand Prix
2,841,153,3,35,17:59:45,26.348,26348,Jaime,Alguersuari,2011,Australian Grand Prix
3,841,30,1,1,17:05:52,25.021,25021,Michael,Schumacher,2011,Australian Grand Prix
4,841,30,2,17,17:32:08,23.988,23988,Michael,Schumacher,2011,Australian Grand Prix


In [ ]:
# Add in some results information
print(pitstop.shape)
pitstop = pitstop.merge(results[['raceId', 'laps', 'statusId']])
print(pitstop.shape)
pitstop.head()

(10089, 11)
(215041, 13)


,raceId,driverId,stop,lap,time,duration,milliseconds,forename,surname,year,name,laps,statusId
0,841,153,1,1,17:05:23,26.898,26898,Jaime,Alguersuari,2011,Australian Grand Prix,58,1
1,841,153,1,1,17:05:23,26.898,26898,Jaime,Alguersuari,2011,Australian Grand Prix,58,1
2,841,153,1,1,17:05:23,26.898,26898,Jaime,Alguersuari,2011,Australian Grand Prix,58,1
3,841,153,1,1,17:05:23,26.898,26898,Jaime,Alguersuari,2011,Australian Grand Prix,58,1
4,841,153,1,1,17:05:23,26.898,26898,Jaime,Alguersuari,2011,Australian Grand Prix,58,1


In [ ]:
# I have some general information to showcase the

In [ ]:
results

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26075,26081,1110,817,213,3,19,16,16,16,0.0,44,+1:43.071,5053521,25,15,1:50.994,227.169,1
26076,26082,1110,858,3,2,18,17,17,17,0.0,44,+1:44.476,5054926,37,9,1:50.486,228.213,1
26077,26083,1110,807,210,27,0,18,18,18,0.0,44,+1:50.450,5060900,26,4,1:49.907,229.415,1
26078,26084,1110,832,6,55,4,\N,R,19,0.0,23,\N,\N,9,19,1:53.138,222.864,130
